In [2]:
import importlib

from german_parser.model.words import WordCNN
from german_parser.model.words import WordEmbedding

from torch.utils.data import DataLoader, default_collate

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import pickle
import torch

In [3]:
from german_parser.util.const import CONSTS

In [4]:
from german_parser.util.dataloader import TigerDatasetGenerator

In [5]:
g = TigerDatasetGenerator(f"{CONSTS['data_dir']}/tiger/tiger_2.2_utf8.xml", (0.2, 0.1), prop_of_tiger_to_use=0.1)

INFO:model:Parsing dataset from '/home/james/programming/ml/german/german_parser/util/../../data/tiger/tiger_2.2_utf8.xml'...
INFO:model:Parsed 5047 sentences.
INFO:model:Generating trees...
INFO:model:4361 (86.41%) trees generated.
INFO:model:Dataset split into 3052 training, 872 dev, and 437 test trees.


In [6]:
train_dataset = g.get_training_dataset()

In [7]:
def my_fn(x):
    res = default_collate(x)
    sentence_lenghts: torch.Tensor = res[1]
    arg_sort = sentence_lenghts.argsort(descending=True)

    return [r[arg_sort] for r in res]

In [8]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=my_fn)

In [10]:
from german_parser.model import TigerModel

In [11]:
m = TigerModel(TigerModel.WordEmbeddingParams(char_set=g.character_set, char_flag_generators=g.character_flag_generators, char_internal_embedding_dim=10, char_part_embedding_dim=10, word_part_embedding_dim=10, char_internal_window_size=3, word_dict=g.inverse_word_dict), TigerModel.LSTMParams(hidden_size=10, bidirectional=True), TigerModel.LSTMParams(hidden_size=10, bidirectional=False))

In [12]:
input = next(iter(train_dataloader))

In [13]:
self_attention, labels, indices = m((input[0], input[1]), train_dataset.get_new_words_dict())

In [14]:
T = input[1][0]

In [15]:
input[1]

tensor([72, 59, 32, 31, 27, 26, 25, 24, 24, 23, 21, 19, 19, 18, 17, 17, 16, 16,
        15, 14, 14, 14, 11, 11, 10, 10, 10,  9,  7,  1,  1,  1])

In [17]:
import torch.nn.functional as F
F.cross_entropy(self_attention[indices], input[2][:,0:T][indices]).backward()